In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("dark")
plt.rcParams['figure.figsize'] = 16, 12
import pandas as pd
from tqdm import tqdm_notebook
import io
from PIL import Image
from glob import glob
from collections import defaultdict
import os
import pickle
import cv2
import math
import json
import itertools as it

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim

from kaggle_camera_model_id_lib.utils import PechkaBot, ImageList, NpzFolder, NCrops, TifFolder, MultiDataset
from kaggle_camera_model_id_lib.models import VggHead, StyleVggHead, IEEEfcn, ResNetFC, ResNetX, FatNet1
from kaggle_camera_model_id_lib.models import InceptionResNetV2fc, InceptionResNetV2fcSmall
from kaggle_camera_model_id_lib.utils import jpg_compress, equalize_v_hist, hsv_convert
from kaggle_camera_model_id_lib.utils import scale_crop_pad, gamma_correction
from kaggle_camera_model_id_lib.utils import patch_quality_dich, n_random_crops, n_pseudorandom_crops
from kaggle_camera_model_id_lib.models import DANet, ResNetFeatureExtractor, AvgFcClassifier, FCDiscriminator
from kaggle_camera_model_id_lib.models import AvgClassifier

import imgaug as ia
from imgaug import augmenters as iaa

In [2]:
crop_center_size = 1520
crop_size = 256
random_crop = transforms.RandomCrop(crop_size)
center_crop = transforms.CenterCrop(crop_center_size)
rvf = transforms.RandomVerticalFlip()
rhf = transforms.RandomHorizontalFlip()
random_flip = lambda img: rvf(rhf(img))

n_crops_search_train = 36
n_crops_train = 9

to_tensor = transforms.ToTensor()
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

In [3]:
train_list_path = '/home/mephistopheies/storage2/data/camera-model-id/train.tsv'
val_path = '/home/mephistopheies/storage2/data/camera-model-id/val/'
test_path = '/home/mephistopheies/storage2/data/camera-model-id/raw/test/'

In [2]:
X_var = Variable(torch.from_numpy(np.random.uniform(-1, 1, size=(5, 3, 256, 256)).astype(np.float32)))

In [3]:
n_classes = 10

model = ResNetX(models.resnet.BasicBlock, [3, 4, 6, 3], num_classes=n_classes, load_resnet='resnet34', pool_type='max')

In [4]:
model(X_var)

Variable containing:
-0.4823  0.5555  0.4618 -0.0635  0.0918 -0.0305  0.1617  0.1392 -0.3897  0.6077
 0.4148 -0.8352  0.0781  0.8591  0.4368  0.1339  0.8367 -0.2658 -0.5379  0.3643
-0.3136  0.2684 -0.2384 -0.0042 -0.0096 -0.2278  0.3131  0.1105 -1.0383  0.1353
-0.0532 -0.1893 -0.6957 -0.2239  0.2679 -0.7076  0.6580  0.2281 -0.0139  0.7205
 0.1965 -0.8259 -0.5584  0.2421 -0.1087 -0.2382 -0.2193 -0.2466 -0.0564 -0.4224
[torch.FloatTensor of size 5x10]